In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from args_reader import read_args
from scicrunch_client import scicrunch
import sys
import time
from interlex_sql import interlex_sql
import re
from collections import defaultdict

args = read_args(api_key='../production_api_scicrunch_key.txt', engine_key='../production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.engine_key)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.engine_key)

In [6]:
engine = create_engine(args.engine_key)
data =  """
        SELECT c.prefix, c.namespace
        FROM curie_catalog as c
        ORDER BY c.namespace
        """
df = pd.read_sql(data, engine)
for i, row in df.iterrows():
    print(row.prefix)

NLXWIKI
BIRNOBI
BIRNOBO
BIRNANN
NIFQUAL
NIFBE
OBOANN
QUALBB
NIFANN
NIFCELL
NIFCHEM
NIFERO
NIFGOCC
NIFGA
NIFMOLROLE
NIFMOLINF
NIFMOL
NIFNCBI
NIFNEURBR
NIFNEURCIR
NIFNEURMOR
NIFNEURNT
NIFORG
NIFSUB
NIFUNCL
NIFNEURON
SAOCORE
NIFGG
NIFINV
NIFRES
NIFSCID
NIFDYS
NIFFUN
NIFNCBISLIM
NEMO
NEMOANN
OBO
BFO
OLD_CHEBI
CLO
CL
DOID
ERO
GO
HP
IAO
NCBITaxon
PATO
PR
PW 
RO
OLD_SO
SO
UBERON
UO
FMA
SIO
ILX

COGAT
EFO
OIO
BFO1


In [7]:
df

,prefix,namespace
0,NLXWIKI,http://neurolex.org/wiki/
1,BIRNOBI,http://ontology.neuinfo.org/NIF/Backend/BIRNLe...
2,BIRNOBO,http://ontology.neuinfo.org/NIF/Backend/BIRNLe...
3,BIRNANN,http://ontology.neuinfo.org/NIF/Backend/BIRNLe...
4,NIFQUAL,http://ontology.neuinfo.org/NIF/Backend/NIF-Qu...
5,NIFBE,http://ontology.neuinfo.org/NIF/Backend/nif_ba...
6,OBOANN,http://ontology.neuinfo.org/NIF/Backend/OBO_an...
7,QUALBB,http://ontology.neuinfo.org/NIF/Backend/qualit...
8,NIFANN,http://ontology.neuinfo.org/NIF/BiomaterialEnt...
9,NIFCELL,http://ontology.neuinfo.org/NIF/BiomaterialEnt...
